In [1]:
import pandas as pd

In [2]:
df_train = pd.read_pickle('train.pkl')
df_test = pd.read_pickle('test.pkl')

In [3]:
#df_train['Electric range (km)'].fillna(0, inplace=True)
#df_test['Electric range (km)'].fillna(0, inplace=True)

df_train["Fuel consumption "][df_train['Electric range (km)'] > 0 ].fillna(0, inplace=True)
df_test["Fuel consumption "][df_test['Electric range (km)'] > 0 ].fillna(0, inplace=True)

In [4]:
ID = df_test['ID']

In [5]:
na_percentage = (df_train.isna().mean() * 100).round(2)

# Sélectionner les colonnes avec 80% ou plus de valeurs manquantes
columns_with_high_na = na_percentage[na_percentage >= 33]

df_train = df_train.drop(columns=columns_with_high_na.index)

na_percentage = (df_test.isna().mean() * 100).round(2)

# Sélectionner les colonnes avec 80% ou plus de valeurs manquantes
columns_with_high_na = na_percentage[na_percentage >= 33]

df_test = df_test.drop(columns=columns_with_high_na.index)

In [6]:
fillna_categorical_columns = ['VFN', 'Mp', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct', 'Fm']

for column in fillna_categorical_columns:
    mode_value = df_train[column].mode()[0]
    df_train[column].fillna(mode_value, inplace=True)

fillna_categorical_columns = ['VFN', 'Mp', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct', 'Ft','Fm']
for column in fillna_categorical_columns:
    mode_value = df_test[column].mode()[0]
    df_test[column].fillna(mode_value, inplace=True)

columns_to_drop = ['Date of registration','r','Status','ID']
df_train = df_train.drop(columns=columns_to_drop)

columns_to_drop = ['Date of registration','r','Status','ID']
df_test = df_test.drop(columns=columns_to_drop)

In [7]:
numeric_columns_with_missing_values = df_train.select_dtypes(include=['float64']).columns[df_train.select_dtypes(include=['float64']).isna().any()].tolist()

# Utiliser la médiane pour remplacer les valeurs manquantes dans les colonnes numériques
df_train[numeric_columns_with_missing_values] = df_train[numeric_columns_with_missing_values].apply(lambda x: x.fillna(x.median()), axis=0)

numeric_columns_with_missing_values = df_test.select_dtypes(include=['float64']).columns[df_test.select_dtypes(include=['float64']).isna().any()].tolist()

# Utiliser la médiane pour remplacer les valeurs manquantes dans les colonnes numériques
df_test[numeric_columns_with_missing_values] = df_test[numeric_columns_with_missing_values].apply(lambda x: x.fillna(x.median()), axis=0)

In [8]:
target = df_train['Ewltp (g/km)']

In [9]:
categorical_columns = ['VFN', 'Mp', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'Ft', 'Fm']
selected_categorical_columns_train = [col for col in categorical_columns if df_train[col].nunique() < 30]

df_one_hot_train = pd.get_dummies(df_train[selected_categorical_columns_train], drop_first=True)

selected_categorical_columns_test = [col for col in categorical_columns if df_test[col].nunique() < 30]

df_one_hot_test = pd.get_dummies(df_test[selected_categorical_columns_test], drop_first=True)

df_train = pd.concat([df_train, df_one_hot_train], axis=1)
df_test = pd.concat([df_test, df_one_hot_test], axis=1)

In [10]:
missing_columns_test = set(df_one_hot_train.columns) - set(df_one_hot_test.columns)
missing_columns_test = [col for col in missing_columns_test if col in df_test.columns]

df_test.drop(columns=missing_columns_test, inplace=True)

columns_to_drop_train = set(df_train.columns) - set(df_test.columns)
df_train.drop(columns=columns_to_drop_train, inplace=True)

columns_to_drop_test = set(df_test.columns) - set(df_train.columns)
df_test.drop(columns=columns_to_drop_test, inplace=True)

In [11]:
df_train['Ewltp (g/km)'] = target

In [12]:
from category_encoders import CountEncoder

categorical_columns_with_values = df_train.select_dtypes(include=['object']).columns[df_train.select_dtypes(include=['object']).nunique() > 1].tolist()

# Initialisez l'encodeur Count
count_encoder = CountEncoder(cols=categorical_columns_with_values)

# Appliquez Count Encoding sur le jeu de données d'entraînement
df_encoded_train = count_encoder.fit_transform(df_train.drop('Ewltp (g/km)', axis=1))


c:\Users\NEIL\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
# Appliquez Count Encoding sur le jeu de données de test
df_encoded_test = count_encoder.transform(df_test)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

# 1. Diviser les données d'entraînement
X_train, X_val, y_train, y_val = train_test_split(df_encoded_train, target, test_size=0.2, random_state=42)
xgb.set_config(verbosity=2)

# 2. Créer une instance du Random Forest Regressor
xgb_model = xgb.XGBRegressor(
    n_estimators=4000,
    max_depth=35,  
    learning_rate=0.005, 
    colsample_bytree=0.80,  
    gamma=10,  
    reg_alpha=0.8,
    reg_lambda=0.1,
    objective='reg:squarederror', 
    tree_method='hist', n_jobs=-1)

# 3. Entraîner le modèle
xgb_model.fit(X_train, y_train)

# 4. Évaluer le modèle sur l'ensemble de validation
y_val_pred = xgb_model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
print('Mean Absolute Error on validation set:', mae_val)

# 5. Faire des prédictions sur les données réelles (selected_data2)
y_data2_pred_xgb = xgb_model.predict(df_encoded_test)

Mean Absolute Error on validation set: 2.816720820942807


In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 1. Split the training data
X_train, X_val, y_train, y_val = train_test_split(df_encoded_train, target, test_size=0.2, random_state=42)

# 2. Create an instance of the Random Forest Regressor
rf_model = RandomForestRegressor(
    random_state=42,
    n_jobs=-1, verbose=10 )

# 3. Train the model
rf_model.fit(X_train, y_train)

# 4. Evaluate the model on the validation set
y_val_pred = rf_model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
print('Mean Absolute Error on validation set:', mae_val)

# 5. Make predictions on the actual data (selected_data2)
y_data2_pred_rf = rf_model.predict(df_encoded_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.2min


building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 11.6min


...

[Parallel(n_jobs=-1)]: Done  80 out of 100 | elapsed: 32.0min remaining:  8.0min


building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  91 out of 100 | elapsed: 36.9min remaining:  3.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 37.8min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    3.8s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    6.4s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    8.1s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:   10.9s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:   20.8s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:   27.4s
[Parallel(n_jobs=16)]: Done  80 out of 100 | elapsed:   31.4s remaining:    7.8s
[Parallel(n_jobs=16)]: Done  91 out of 100 | elapsed:   33.8s remaining:    3.3s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   35.4s finished


Mean Absolute Error on validation set: 2.8855988604112968


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    3.8s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    5.7s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    7.0s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    7.9s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:   10.1s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:   15.1s
[Parallel(n_jobs=16)]: Done  80 out of 100 | elapsed:   18.1s remaining:    4.4s
[Parallel(n_jobs=16)]: Done  91 out of 100 | elapsed:   19.5s remaining:    1.8s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   20.8s finished


In [23]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 1. Split the training data
X_train, X_val, y_train, y_val = train_test_split(df_encoded_train, target, test_size=0.2, random_state=42)

# 2. Create an instance of the LightGBM Regressor
lgbm_model = LGBMRegressor(n_estimators=4000,
    random_state=42,
    n_jobs=-1,
    verbose=10
)

# 3. Train the model
lgbm_model.fit(X_train, y_train)

# 4. Evaluate the model on the validation set
y_val_pred = lgbm_model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
print('Mean Absolute Error on validation set:', mae_val)

# 5. Make predictions on the actual data (selected_data2)
y_data2_pred_lgbm = lgbm_model.predict(df_encoded_test)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.958706
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.230125
[LightGBM] [Debug] init for col-wise cost 0.064619 seconds, init for row-wise cost 0.164763 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.116809 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 3517
[LightGBM] [Info] Number of data points in the train set: 6057319, number of used features: 49
[LightGBM] [Info] Start training from score 340.302066
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [D

In [45]:
import numpy as np

# Initialize the best MAE and best weight
best_mae = np.inf
best_weight = None

# Iterate over a range of weights from 0 to 1 in steps of 0.01
for weight in np.arange(0, 1.01, 0.01):
    # Calculate the weighted prediction
    y_val_pred_weighted = y_val_pred_xgb * weight + y_val_pred_rf * (1 - weight) + y_val_pred_lgbm * (1 - weight)
    

    # Calculate the MAE
    mae_val = mean_absolute_error(y_val, y_val_pred_weighted)
    print('Weight:', weight)
    print('Mean Absolute Error:', mae_val)
    # If this MAE is better than the best MAE so far, update the best MAE and best weight
    if mae_val < best_mae:
        best_mae = mae_val
        best_weight = weight

print('Best weight for XGBoost:', best_weight)
print('Best Mean Absolute Error:', best_mae)

Weight: 0.0
Mean Absolute Error: 2.8855988604112968
Weight: 0.001
Mean Absolute Error: 2.885382503254887
Weight: 0.002
Mean Absolute Error: 2.8851664419382743
Weight: 0.003
Mean Absolute Error: 2.8849507034970223
Weight: 0.004
Mean Absolute Error: 2.8847352897296643
Weight: 0.005
Mean Absolute Error: 2.884520289195996
Weight: 0.006
Mean Absolute Error: 2.884305606626303
Weight: 0.007
Mean Absolute Error: 2.88409121151324
Weight: 0.008
Mean Absolute Error: 2.8838771290028724
Weight: 0.009000000000000001
Mean Absolute Error: 2.8836634127318046
Weight: 0.01
Mean Absolute Error: 2.8834500295800995
Weight: 0.011
Mean Absolute Error: 2.883237217800254
Weight: 0.012
Mean Absolute Error: 2.8830246767840895
Weight: 0.013000000000000001
Mean Absolute Error: 2.8828123986172987
Weight: 0.014
Mean Absolute Error: 2.882600413948329
Weight: 0.015
Mean Absolute Error: 2.88238869278555
Weight: 0.016
Mean Absolute Error: 2.8821774154033477
Weight: 0.017
Mean Absolute Error: 2.8819666031703597
Weight: 0.

In [56]:
y_val_pred_lgbm = lgbm_model.predict(X_val)


In [55]:
y_val_pred_cat = cat_model.predict(X_val)


In [36]:
# 4. Evaluate the model on the validation set
y_val_pred_rf = rf_model.predict(X_val)
y_val_pred_xgb = xgb_model.predict(X_val)

y_val_pred_65 = y_val_pred_xgb * 0.65 + y_val_pred_rf * 0.35
mae_val = mean_absolute_error(y_val, y_val_pred_65)
print('Mean Absolute Error on validation set 75% xgb:', mae_val)


y_val_pred_743 = y_val_pred_xgb * 0.743 + y_val_pred_rf * 0.257
mae_val = mean_absolute_error(y_val, y_val_pred_743)
print('Mean Absolute Error on validation set 743% xgb:', mae_val)


y_val_pred_75 = y_val_pred_xgb * 0.75 + y_val_pred_rf * 0.25
mae_val = mean_absolute_error(y_val, y_val_pred_75)
print('Mean Absolute Error on validation set 75% xgb:', mae_val)

y_val_pred_85 = y_val_pred_xgb * 0.85 + y_val_pred_rf * 0.15
mae_val = mean_absolute_error(y_val, y_val_pred_85)
print('Mean Absolute Error on validation set 85% xgb:', mae_val)


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    5.5s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:   10.9s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:   17.3s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:   20.9s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:   29.8s
[Parallel(n_jobs=16)]: Done  80 out of 100 | elapsed:   37.1s remaining:    9.2s
[Parallel(n_jobs=16)]: Done  91 out of 100 | elapsed:   40.7s remaining:    3.9s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   43.0s finished


Mean Absolute Error on validation set 75% xgb: 2.8070470843773103
Mean Absolute Error on validation set 85% xgb: 2.808652016282


In [49]:
mean_pred = pd.concat([pd.DataFrame(y_data2_pred_xgb) * 0.743 + pd.DataFrame(y_data2_pred_rf) * 0.257],axis=1).mean(axis=1)
len(mean_pred)

1892913

In [34]:
mean_pred = pd.concat([pd.DataFrame(y_data2_pred_xgb) * 0.75 + pd.DataFrame(y_data2_pred_rf) * 0.25],axis=1).mean(axis=1)
submission_df = pd.DataFrame({'ID': ID, 'Ewltp (g/km)': mean_pred})

# Enregistrer le DataFrame dans un fichier CSV
submission_df.to_csv(r'submission_ensemble_mean_pond_75percent.csv', index=False)

In [35]:
mean_pred = pd.concat([pd.DataFrame(y_data2_pred_xgb) * 0.85 + pd.DataFrame(y_data2_pred_rf) * 0.15],axis=1).mean(axis=1)
submission_df = pd.DataFrame({'ID': ID, 'Ewltp (g/km)': mean_pred})

# Enregistrer le DataFrame dans un fichier CSV
submission_df.to_csv(r'submission_ensemble_mean_pond_85percent.csv', index=False)

In [19]:
median_pred = pd.concat([pd.DataFrame(y_data2_pred_xgb), pd.DataFrame(y_data2_pred_rf)],axis=1).median(axis=1)
len(mean_pred)

1892913

In [33]:
pd.DataFrame(mean_pred).reset_index(drop=True)

,0
0,94.297607
1,385.659707
2,385.659707
3,385.659707
4,385.659707
...,...
1892908,93.021899
1892909,-0.059540
1892910,-0.195818
1892911,354.670022


In [16]:
submission_df = pd.DataFrame({'ID': ID, 'Ewltp (g/km)': y_data2_pred_xgb})

# Enregistrer le DataFrame dans un fichier CSV
submission_df.to_csv(r'submission_xgb.csv', index=False)

In [50]:
submission_df = pd.DataFrame({'ID': ID, 'Ewltp (g/km)': mean_pred})

# Enregistrer le DataFrame dans un fichier CSV
submission_df.to_csv(r'submission_ensemble_mean_743_xgb.csv', index=False)

In [21]:
submission_df = pd.DataFrame({'ID': ID, 'Ewltp (g/km)': median_pred})

# Enregistrer le DataFrame dans un fichier CSV
submission_df.to_csv(r'submission_ensemble_median.csv', index=False)

In [81]:
submission_df

,ID,Ewltp (g/km)
0,8000000,94.311741
1,8000001,385.652042
2,8000002,385.652042
3,8000003,385.652042
4,8000004,385.652042
...,...,...
1892908,9892908,92.396541
1892909,9892909,0.137040
1892910,9892910,-0.419923
1892911,9892911,354.657145
